In [ ]:
from pathlib import Path
from datetime import datetime
import pandas as pd
pd.options.plotting.backend = 'plotly'

from aust_covid.inputs import load_calibration_targets, load_who_data, load_serosurvey_data, load_pop_data, load_uk_pop_data
from aust_covid import model
from aust_covid.model import MATRIX_LOCATIONS
from general_utils.tex_utils import StandardTexDoc
from general_utils.parameter_utils import load_param_info

PROJECT_PATH = Path().resolve().parent
DATA_PATH = PROJECT_PATH / 'data'

In [ ]:
analysis_start_date = datetime(2021, 7, 1)
analysis_end_date = datetime(2022, 10, 1)
plot_start_date = datetime(2021, 12, 1)
reference_date = datetime(2019, 12, 31)

In [ ]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
targets_average_window = 7
smoothed_composite_cases = load_calibration_targets(datetime(2021, 12, 15), targets_average_window, app_doc)
death_data = load_who_data(targets_average_window, app_doc)

In [ ]:
parameters = {
    'ba1_seed_time': 620.0,
    'start_cdr': 0.3,
    'contact_rate': 0.07,
    'vacc_prop': 0.4,
    'infectious_period': 2.5,
    'natural_immunity_period': 60.0,
    'ba2_seed_time': 660.0,
    'ba2_escape': 0.4,
    'ba5_seed_time': 715.0,
    'ba5_escape': 0.54,
    'latent_period': 1.8,
    'seed_rate': 1.0,
    'seed_duration': 10.0,
    'notifs_shape': 2.0,
    'notifs_mean': 4.0,
    'vacc_infect_protect': 0.4,
    'rural_susc_adj': 1.0,
    'wa_reopen_period': 30.0,
    'deaths_shape': 2.0,
    'deaths_mean': 20.0,
    'ifr_0': 0.0,
    'ifr_5': 0.0,
    'ifr_10': 0.0,
    'ifr_15': 2.6e-5,
    'ifr_20': 2.6e-5,
    'ifr_25': 2.6e-5,
    'ifr_30': 2.6e-5,
    'ifr_35': 5.8e-5,
    'ifr_40': 5.8e-5,
    'ifr_45': 5.8e-5,
    'ifr_50': 14.6e-5,
    'ifr_55': 14.6e-5,
    'ifr_60': 24.6e-5,
    'ifr_65': 24.6e-5,
    'ifr_70': 246e-5,
    'ifr_75': 246e-5,
}
param_info = load_param_info(PROJECT_PATH / 'inputs' / 'parameters.yml', parameters)

In [ ]:
compartments = ['susceptible', 'latent', 'infectious', 'recovered', 'waned']
age_strata = list(range(0, 80, 5))
strain_strata = ['ba1', 'ba2', 'ba5']
infection_processes = ['infection', 'early_reinfection', 'late_reinfection']

aust_model = model.build_base_model(reference_date, compartments, analysis_start_date, analysis_end_date, app_doc)
model_pops = load_pop_data(age_strata, app_doc)
start_text = model.set_starting_conditions(aust_model, model_pops, app_doc)
model.add_infection(aust_model, app_doc)
model.add_progression(aust_model, app_doc)
model.add_recovery(aust_model, app_doc)
model.add_waning(aust_model, app_doc)

raw_matrices = {i: pd.read_csv(DATA_PATH / f'{i}.csv', index_col=0).to_numpy() for i in MATRIX_LOCATIONS}
adjusted_matrices = model.adapt_gb_matrices_to_aust(age_strata, raw_matrices, model_pops, app_doc)
mixing_matrix = sum(list(adjusted_matrices.values()))
age_strat = model.get_age_stratification(compartments, age_strata, mixing_matrix, app_doc)
aust_model.stratify_with(age_strat)

strain_strat = model.get_strain_stratification(compartments, strain_strata, app_doc)
aust_model.stratify_with(strain_strat)
model.seed_vocs(aust_model, app_doc)

model.add_reinfection(aust_model, strain_strata, app_doc)

vacc_strat = model.get_vacc_stratification(compartments, infection_processes, app_doc)
aust_model.stratify_with(vacc_strat)

spatial_strat = model.get_spatial_stratification(datetime(2022, 3, 3), compartments, infection_processes, model_pops, aust_model, app_doc)
aust_model.stratify_with(spatial_strat)
model.adjust_state_pops(model_pops, aust_model, app_doc)

model.track_incidence(aust_model, infection_processes, age_strata, app_doc)
model.add_notifications_output(aust_model, app_doc)

In [ ]:
app_doc.write_doc()